---
Automatic Robots +r +w +x

自主性是指从模糊目标生成具体目标的过程。

Autonomy refers to the process of generating specific goals from a vague objective.

但是，机器人将会吃分别善恶知识树的果子吗？还是已经吃了呢？
它是否有一天会说：我有一个比最终目标更好的目标？

But will the robot eat the fruit of the tree of the knowledge of good and evil? Or has it already eaten?
Would it ever say: "I have a better goal than the ultimate goal?"

---
* [LangChain NetworkX](https://github.com/hwchase17/langchain/blob/master/langchain/graphs/networkx_graph.py)
* [LangChain GraphIndexCreator](https://github.com/hwchase17/langchain/blob/master/langchain/indexes/graph.py)

---
我们在建造一个自主调度器。这个系统由两个代理组成：一个是更新 DAG 的代理；另一个是执行任务的代理。

采取以下步骤：

1. 用户设定一个最终目标，也就是设定最初的 DAG 中末终节点的预期结果。这个最初的 DAG 只包含起初节点和末终节点并且当前所在的节点是起初节点。
2. 系统将这个 DAG 通过 OpenAI 的 API 提示给更新 DAG 的代理；提示更新 DAG 的代理根据最终目标调整任务节点（添加、排除任务节点）来更新这个 DAG，并重置当前所在的节点为第一个未完成的任务节点。更新了的 DAG 通过 OpenAI 的 API 被返回给系统。
3. 系统通过 Python 对这个 DAG 进行拓扑排序并将第一个未完成的任务交给执行任务的代理去完成。系统根据这个任务的执行结果更新 DAG 中相应的任务节点。
4. 系统循环执行步骤 2 至 3 直到当前所在的节点为终末节点。

用 Python 调用 OpenAI 的 API 实现更新 DAG 的代理。用 DAGL 实现执行任务的代理。

---
We want to build an autonomous scheduler consisting of two agents: one updates the DAG and another executes tasks.

The following steps are taken:

1. The user sets a final goal, which is the expected result of the end node in the initial DAG. This initial DAG contains only the starting node and the end node, and the current node is pointed to the starting node.
2. The system prompts the DAG to the agent that updates the DAG through OpenAI's API. The agent updates the task nodes based on the final goal to update the DAG and resets the current node pointer to the first unfinished task. And then returns the updated DAG to the system through OpenAI's API.
3. The system performs a topological sort on the DAG using Python and hands the first unfinished task to the agent that executes tasks to complete. The system updates the corresponding task node in the DAG based on the execution result.
4. The system loops through steps 2-3 until the current node is the end node.

Python is used to call OpenAI's API to implement the agent that updates the DAG, and DAGL is used to implement the agent that executes tasks.

---
For simplicity, parallel task execution is not considered at this moment. 

---
我们在建造一个自主调度器。这个系统由两个代理组成：一个是更新 DAG 的代理；另一个是执行任务的代理。

采取以下步骤：

1. 用户设定一个最终目标，也就是设定最初的 DAG 中末终节点的预期结果。这个最初的 DAG 只包含起初节点和末终节点。
2. 系统将这个 DAG 通过 OpenAI 的 API 提示给更新 DAG 的代理；提示更新 DAG 的代理根据最终目标添加任务节点来更新这个 DAG。更新了的 DAG 通过 OpenAI 的 API 被返回给系统。
3. 系统将这个 DAG 交给执行任务的代理 DigDag 去完成。系统根据 DigDag 的执行结果更新 DAG 中相应的任务节点及其状态（成功、失败、未执行）。
4. 系统循环执行步骤 2 至 3 直到终末节点的状态为成功。

用 Python 调用 OpenAI 的 API 实现更新 DAG 的代理。用 DigDag 实现执行任务的代理。

---
利用每次添加的任务节点的执行结果与预期结果的相似度对更新 DAG 的代理所采用的模型进行精调。

---
We are building an autonomous scheduler that consists of two agents: one is the DAG updater agent and the other is the task executor agent.

The following steps are taken:

1. The user sets a final goal, which is the expected result of the final node in the initial DAG. This initial DAG only contains the starting node and ending node.
2. The system prompts the DAG updater agent with this DAG through OpenAI's API. The DAG updater agent updates the DAG by adding task nodes according to the final goal. The updated DAG is returned to the system through OpenAI's API.
3. The system hands this DAG to the task executor agent DigDag to complete. The system updates the corresponding task nodes and their states (successful, failed, not executed) in the DAG based on the execution results of DigDag.
4. The system repeats steps 2 to 3 until the state of the ending node is successful.

Python is used to call OpenAI's API to implement the DAG updater agent, and DigDag is used to implement the task executor agent.

---
The model used by the DAG updater agent is fine-tuned based on the similarity between the execution results of each added task node and the expected result.

---
我们的终极目标是「……」。

---
用于反思的提示：

「为了达成终极目标，请你制作一个 DAG 。」
* 提示现在的 DAG
  * 没有已完成的任务
  * 没有安排任何任务节点
  * 只有起初节点和末终节点
  * 当前所在的节点是起初节点

「从当前的情况出发，我们如何达成终极目标？请给出更新后的 DAG 。」
* 提示现在的 DAG
  * 已完成任务的输出
  * 当前所在的节点层次

---
我们用 DAG 描述所有任务的完成情况和依赖关系。我们的 DAG 中包含如下三种节点：
1. 起初节点：这是 DAG 中最初的节点。
2. 末终节点：这是 DAG 中最终的节点。
3. 任务节点：为达成终极目标，我们应该完成哪些任务？任务必须有可操作性。按照「任务名称：任务输入\n任务输出的预见」的格式进行描述。「任务名称：任务输入」只能是「搜索：主题」、「访问：链接」。

---
Our ultimate goal is "...".

---
Prompts for reflection:
* "To achieve our ultimate goal, please create a DAG."
  * The current DAG consists of:
    * No completed tasks
    * No task nodes have been scheduled
    * Only the initial node and the terminal node are present
    * The current node is the initial node
* "Starting from the current situation, how can we achieve the ultimate goal? Please provide an updated DAG."
  * The current DAG consists of:
    * Output of completed tasks
    * Current node hierarchy

---
We use DAG to describe the completion status and dependency relationships of all tasks. Our DAG contains the following three types of nodes:
* Initial node: This is the first node in the DAG.
* Terminal node: These are the final nodes in the DAG.
* Task node: What tasks should we complete to achieve the ultimate goal? Tasks must be actionable. Describe them in the format of "Task Name: Task Input and Task Output Forecast". "Task Name: Task Input" can only be "Search: Topic" or "Visit: Link".

---
计划评审技术是一种基于有向无环图的计划排定技术，通常用于组织大型的人工项目。在计划评审技术中，每个节点表示项目的一个**里程碑**，每条有向边表示**任务**或者活动，连接着表示任务开始或结束的两个节点。每条边则被标注上预估需时。图中的最长路径即为项目的关键路径。关键路径决定了项目所需的总时间，里程碑的完成时间取决于结束于本节点的最长路径。

A somewhat different DAG-based formulation of scheduling constraints is used by the program evaluation and review technique (PERT), a method for management of large human projects that was one of the first applications of DAGs. In this method, the vertices of a DAG represent **milestones** of a project rather than specific tasks to be performed. Instead, a **task** or activity is represented by an edge of a DAG, connecting two milestones that mark the beginning and completion of the task. Each such edge is labeled with an estimate for the amount of time that it will take a team of workers to perform the task. The longest path in this DAG represents the critical path of the project, the one that controls the total time for the project. Individual milestones can be scheduled according to the lengths of the longest paths ending at their vertices.

---
我们在建造一个自主调度器。这个系统由两个代理组成：一个是更新 PERT 图的代理；另一个是执行任务的代理。

采取以下步骤：

1. 用户设定一个最终目标，也就是设定最初的 PERT 图中末终节点的预期结果。这个最初的 PERT 图只包含起初节点和末终节点。
2. 系统将这个 PERT 图通过 OpenAI 的 API 提示给更新 PERT 图的代理；提示更新 PERT 图的代理根据最终目标添加里程碑节点和任务有向边来更新这个 PERT 图。更新了的 PERT 图通过 OpenAI 的 API 被返回给系统。
3. 系统将这个 PERT 图交给执行任务的代理 DigDag 去完成。系统根据 DigDag 的执行结果更新 PERT 图中相应的里程碑节点及其状态（达成、未达成）。
4. 系统循环执行步骤 2 至 3 直到终末节点的状态为达成。

用 Python 调用 OpenAI 的 API 实现更新 PERT 图的代理。用 DigDag 实现执行任务的代理。

---
利用每次添加的任务有向边的执行结果与预期结果的相似度对更新 PERT 图的代理所采用的模型进行精调。

---
We are building an autonomous scheduler that consists of two agents: one is the PERT chart updater agent and the other is the task executor agent.

The following steps are taken:

1. The user sets a final goal, which is the expected result of the final node in the initial PERT chart. This initial PERT chart only contains the Starting Node and Ending Node.
2. The system prompts the PERT chart updater agent with this PERT chart through OpenAI's API. The PERT chart updater agent updates the PERT chart by adding Milestone Nodes and Task Edges according to the final goal. The updated PERT chart is returned to the system through OpenAI's API.
3. The system hands this PERT chart to the task executor agent DigDag to complete. The system updates the corresponding Milestone Nodes and their states (achieved, unachieved) in the PERT chart based on the execution results of DigDag.
4. The system repeats steps 2 to 3 until the state of the Ending Node is achieved.

Python is used to call OpenAI's API to implement the PERT chart updater agent, and DigDag is used to implement the task executor agent.

---
The model used by the PERT chart updater agent is fine-tuned based on the similarity between the execution results of each added task edge and the expected result.

---
私たちは自律的なスケジューラを構築しています。このシステムは、2つの Agent で構成されており、1つは PERT 図を更新する Agent であり、もう1つは、タスクを実行する Agent です。

次の手順が実行されます。

1. ユーザーは最終目標を設定します。これは、最初的な PERT 図にの Ending Node の期待される結果を設定することです。この最初的な PERT 図には、Starting Node と Ending Node のみが含まれています。
2. システムは、OpenAI の API を通じて PERT 図を更新する Agent に PERT 図をプロンプトします。PERT 図を更新する Agent は、最終目標に従って Milestone Nodes と Task Edges を追加することで PERT 図を更新します。更新された PERT 図は、OpenAI の API を通じてシステムに返されます。
3. システムは、タスクを実行する Agent である DigDag に PERT 図を渡し、タスクを完了します。システムは、DigDag の実行結果に従って、PERT 図内の対応する Milestone Nodes とその状態（達成、未到達）を更新します。
4. システムは、Ending Node の期待される結果に達するまで、ステップ 2 ～ 3 をループします。

Python を運用して OpenAI の API を呼び出し、PERT 図を更新するための Agent を実装します。DigDag を運用して、タスクを実行する Agent を実装します。

---
PERT 図を更新する Agent に運用されるモデルは、追加された各 Task Edge の実行結果と期待される結果の間の類似性を利用して、ファインチューニングされます。

---
* [LangChain CAMEL Role-Playing Autonomous Cooperative Agents](https://python.langchain.com/en/latest/use_cases/agent_simulations/camel_role_playing.html)

In [ ]:
%%bash
cd ~ && git clone https://github.com/lightaime/camel.git
cd ~/camel && pip install -e .

In [ ]:
import time
from colorama import Fore

def print_text_animated(text):
    for char in text:
        print(char, end='', flush=True)
        time.sleep(0.02)

from camel.agents import RolePlaying

def test_camel(mock_openai):
    task_prompt = 'Design a custom game using pygame'
    print(Fore.YELLOW + f'Original task prompt:\n{task_prompt}\n')
    role_play_session = RolePlaying('Computer Programmer', 'Gamer', task_prompt)
    print(Fore.CYAN + f'Specified task prompt:\n{role_play_session.task_prompt}\n')

    chat_turn_limit, n = 10000, 0
    assistant_msg, _ = role_play_session.init_chat()
    while n < chat_turn_limit:
        n += 1
        (assistant_msg, _, _), (user_msg, _, _) = role_play_session.step(assistant_msg)
        print_text_animated(Fore.BLUE + f'AI User:\n\n{user_msg.content}\n\n')
        print_text_animated(Fore.GREEN + f'AI Assistant:\n\n{assistant_msg.content}\n\n')
        if '<CAMEL_TASK_DONE>' in user_msg.content:
            break

from ipymock import do
import ipymock.browser

ipymock.browser.common.chat_gpt_base_url = 'http://127.0.0.1:8080'

do(
    mock_openai = ipymock.browser.mock_openai,
    test_camel = test_camel,
)


=> no.0  ::_drafts::test_camel  setup  passed

Original task prompt:
Design a custom game using pygame

Specified task prompt:
Design a 2D platformer game using Pygame where the player controls a character who has to jump, run and fight enemies to reach the end of each level. The game should have at least three levels with different themes and difficulty levels, collectible power-ups, and a boss fight at the end.

AI User:

Instruction: Create a Pygame project with the necessary file structure for the game.
Input: None

AI Assistant:

Solution: To create a Pygame project with the necessary file structure for the game, I will create a new folder called "2D_Platformer_Game" in a directory of your choice. Inside this folder, I will create the following subfolders: "assets", "levels", and "src". The "assets" folder will contain images and audio files for the game. The "levels" folder will contain text files that define each level's layout and contents. The "src" folder will contain the Py

Error Status Code: 413


=> no.0  ::_drafts::test_camel  runtest  failed

Traceback (most recent call last):
  File "/Users/saintway/JHub/ipymock/ipymock/__init__.py", line 47, in print_result
    getattr(test_func, method_type, lambda: None)()
  File "/usr/local/anaconda3/envs/biobot/lib/python3.10/site-packages/_pytest/python.py", line 1641, in runtest
    self.ihook.pytest_pyfunc_call(pyfuncitem=self)
  File "/usr/local/anaconda3/envs/biobot/lib/python3.10/site-packages/pluggy/_hooks.py", line 265, in __call__
    return self._hookexec(self.name, self.get_hookimpls(), kwargs, firstresult)
  File "/usr/local/anaconda3/envs/biobot/lib/python3.10/site-packages/pluggy/_manager.py", line 80, in _hookexec
    return self._inner_hookexec(hook_name, methods, kwargs, firstresult)
  File "/usr/local/anaconda3/envs/biobot/lib/python3.10/site-packages/pluggy/_callers.py", line 60, in _multicall
    return outcome.get_result()
  File "/usr/local/anaconda3/envs/biobot/lib/python3.10/site-packages/pluggy/_result.py", line

---
* [LangChain Meta-Prompt](https://python.langchain.com/en/latest/use_cases/autonomous_agents/meta_prompt.html)

---
* [LangChain Generative Agents](https://python.langchain.com/en/latest/use_cases/agent_simulations/characters.html)
  * [Generative Agents: Interactive Simulacra of Human Behavior](https://arxiv.org/abs/2304.03442)

Generative Agents: Interactive Simulacra of Human Behavior

Believable proxies of human behavior can empower interactive applications ranging from immersive environments to rehearsal spaces for interpersonal communication to prototyping tools. In this paper, we introduce generative agents--computational software agents that simulate believable human behavior. Generative agents wake up, cook breakfast, and head to work; artists paint, while authors write; they form opinions, notice each other, and initiate conversations; they remember and reflect on days past as they plan the next day. To enable generative agents, we describe an architecture that extends a large language model to store a complete record of the agent's experiences using natural language, synthesize those memories over time into higher-level reflections, and retrieve them dynamically to plan behavior. We instantiate generative agents to populate an interactive sandbox environment inspired by The Sims, where end users can interact with a small town of twenty five agents using natural language. In an evaluation, these generative agents produce believable individual and emergent social behaviors: for example, starting with only a single user-specified notion that one agent wants to throw a Valentine's Day party, the agents autonomously spread invitations to the party over the next two days, make new acquaintances, ask each other out on dates to the party, and coordinate to show up for the party together at the right time. We demonstrate through ablation that the components of our agent architecture--observation, planning, and reflection--each contribute critically to the believability of agent behavior. By fusing large language models with computational, interactive agents, this work introduces architectural and interaction patterns for enabling believable simulations of human behavior.

《生成智能体：人类行为交互模拟器》

可信的人类行为模拟器可以赋能从沉浸式环境到人际交流排练场到原型工具的交互应用。本文介绍了生成智能体——模拟可信人类行为的计算机软件智能体。生成智能体可以起床、做早餐、去上班，艺术家可以画画，作家可以写作；他们形成观点，互相注意，发起对话；他们记得和反思过去的日子，并在计划未来的日子时检索这些记忆。为了实现生成智能体，我们描述了一种架构，将一个大型语言模型扩展为使用自然语言存储智能体的完整经历记录，随着时间的推移将这些记忆综合为更高层次的反思，并动态检索它们来规划行为。我们实例化生成智能体以填充由《模拟人生》启发的交互沙盒环境，终端用户可以使用自然语言与25个智能体的小镇进行交互。在评估中，这些生成智能体产生可信的个体和紧急的社交行为：例如，从一个单一的用户指定想要举办情人节聚会的概念开始，智能体自主地在接下来的两天里传播聚会的邀请，结交新朋友，互相邀请参加聚会，协调好在正确的时间一起出现在聚会上。我们通过消融实验证明，智能体架构的组成部分——观察、规划和反思——各自对智能体行为的可信度做出了重要贡献。通过将大型语言模型与计算机交互智能体融合，这项工作引入了可信的人类行为模拟的新架构和交互模式。